In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

from google.cloud import bigquery

# Fetch Credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'local-receiver-341803-9e38525e7b40.json'

# Build BigQuery Client
client = bigquery.Client()

In [21]:
#Get the dataset

# Query for Position Histories from BigQuery - change limit after we know it works
QUERY = ('SELECT lastTrainReporting.positionHistoryList FROM `local-receiver-341803.testTrainEvents.train-event-data` LIMIT 50')
query_job = client.query(QUERY)

# Wait for job to finish
rows = query_job.result()

print("Num rows fetched: " + str(rows.total_rows))




Num rows fetched: 50


In [ ]:
# Create the data frame - should have a timestamp in it

# Fetch each position time and milepost
positionMilepostTimesDataFrame = pd.DataFrame()

# For each returned record
for row in rows:
    # Scan each position history
    for positionHistory in row.get('positionHistoryList'):
        # The time that the train was at the specific milepost
        # Record the position time as seconds from epoch to use in regression model
        #positionTime = positionHistory.get('positionTime').timestamp()
        positionTime = positionHistory.get('positionTime')

        # How fast the train is traveling
        speedMPH = positionHistory.get('speedMPH')

        # Milepost location information
        milepostLocation = positionHistory.get('milepostLocation')
        # The unique identifier of the subdivision
        subdivisionId = milepostLocation.get('subdivisionId')
        # The unique identifier fo the milepost
        milepostNumber = milepostLocation.get('milepost').get('milepostNumber')

        # Construct new dataframe record
        dataframeRow = {'PositionTime': positionTime, 'SpeedMPH': speedMPH, 
                        'SubdivisionId': subdivisionId, 'MilepostNumber':milepostNumber}

        # Append the row to the data frame.
        positionMilepostTimesDataFrame = positionMilepostTimesDataFrame.append(dataframeRow, ignore_index = True)


#Return an item and drop from frame
date_time = pd.to_datetime(positionMilepostTimesDataFrame.pop("PositionTime"), format='%Y-%m-%d %H:%M:%S')

print("date time: \n" + str(date_time))

print("cols: " + str(positionMilepostTimesDataFrame.columns))

positionMilepostTimesDataFrame.head()

In [ ]:
#Evolution of Features over time

In [ ]:
# Inspect and clean up - look at statistics of dataset